# Analyse hydrogen financing results from Pypsa-Earth network
This notebook analyze the financing hydrogen data of the network (.nc) file. It generates comprehensive plots and summaries to visualize the results.
* Capital expenditures (CAPEX)
* Operational costs (OPEX)
* Revenue
* Levelized Cost of Hydrogen (LCOH)*

Sources: 
- Statistics module: https://pypsa.readthedocs.io/en/latest/examples/statistics.html
- Templates and colors: https://github.com/pypsa-meets-earth/documentation/blob/main/notebooks/network_analysis.ipynb

File is needed:
* PyPSA network file (.nc) includes the hydrogen financing information.

## Import packages

In [ ]:
import yaml
import pypsa
import warnings
import matplotlib.pyplot as plt
import pandas as pd

## Path settings
This section reads the config parameters from your config.yaml file and automatically reads the output of the optimization with those settings

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
# Set your pypsa-earth directory here
PARENT = "/home/pypsa-earth/"
# Set your network path (.nc) here
network_path = PARENT + f"results/networks/elec_s_10_ec_lc3.0_Co2L_3H_2035_0.071_AB_14export.nc"
n = pypsa.Network(network_path)
config_path = PARENT + f"/config.yaml"
config = yaml.safe_load(open(config_path))